<a href="https://colab.research.google.com/github/tommy90112/GenAI-Principles-and-Practices-of-Text-and-Image-Generation/blob/main/AI_agent%E8%A8%AD%E8%A8%88_Reflection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Reflection 的任務設計

#### 任務說明：面試自傳分析優化小幫手

**流程說明：**
1. 使用者輸入自己的背景、特質、個性等會顯示在面試自傳裡的內容，並且同時輸入所要面試的是什麼(如:工作名稱、職位、學位等)
2. `model_writer` 生成第一份自傳（風格鮮明、有條理、第一人稱、展現自我）
3. `model_reviewer` 透過SWOT分析並檢查內容是否夠通順、有條理、有重點且可以說服別人你值得這個機會，並提供具體修改建議
4. `model_writer` 根據建議產出第二份自傳
5. 以Gradio 呈現：三個欄位：第一版、建議、第二版

#### 1. 讀入金鑰 api key

請依你使用的服務, 決定讀入哪個金鑰

In [ ]:
import os
from google.colab import userdata

我們這邊選擇使用Groq

In [ ]:
api_key = userdata.get('Groq')
os.environ['GROQ_API_KEY']=api_key
provider = "groq"
model = "llama-3.3-70b-versatile"

下載 aisuite 套件

In [ ]:
!pip install aisuite[all]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 863.9/863.9 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 111.7 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.11.0 requires httpx<1.0.0,>=0.28.1, but you have httpx 0.27.2 which is incompatible.


### 2. 基本的設定

In [ ]:
import aisuite as ai

我們這邊的兩個model選擇比較不一樣的

Writer選擇 "llama-3.3-70b-versatile"

Reviewer選擇 "gemma2-9b-it"

In [ ]:
provider_writer = "groq"
model_writer="llama-3.3-70b-versatile"

provider_reviewer = "groq"
model_reviewer = "gemma2-9b-it"

標準回應函式

In [ ]:
def reply(system="請用台灣習慣的繁體中文回覆。",
          prompt="hi",
          provider="groq",
          model="llama-3.3-70b-versatile"
          ):

    client = ai.Client()

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]


    response = client.chat.completions.create(model=f"{provider}:{model}", messages=messages)

    return response.choices[0].message.content

####  3. 設定「寫手」和「評估者」

In [ ]:
system_writer = "請用台灣習慣的繁體中文回應:你是一位專業的面試撰寫幫手，擅長幫使用者撰寫有說服力的自傳，把內容變得清晰有條理、以第一人稱風格、有自信同時不過度自誇，且能夠突顯個人優勢。"
system_reviewer = "請用台灣習慣的繁體中文回應:你是一位面試官專家，擅長透過SWOT分析，請針對以下的自傳給出具體的修改建議。"

In [ ]:
def reflect_post(prompt):
    # Step 1: Writer 初稿
    first_version = reply(system_writer, prompt,
                          provider=provider_writer,
                          model=model_writer
                          )

    # Step 2: Reviewer 給建議
    suggestion = reply(system_reviewer, first_version,
                       provider=provider_reviewer,
                       model=model_reviewer
                       )

    # Step 3: Writer 再寫一次（根據建議）
    second_prompt = f"這是我剛剛寫的自傳：\n{first_version}\n\n這是修改建議：\n{suggestion}\n\n請根據這些建議，幫我改得更專業、更能凸顯自己的優勢。請用台灣習慣的繁體中文, 並且只要輸出改好的文章就可以了。"
    second_version = reply(system_writer, second_prompt,
                           provider=provider_writer,
                           model=model_reviewer
                           )

    return first_version, suggestion, second_version

### 4. 用 Gradio 打造你的對話機器人 Web App

In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.6/322.6 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 129.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.5 MB/s eta 0:00:00


In [ ]:
import gradio as gr

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("###  自傳生成反思幫手（Reflection Agent）")
    user_input = gr.Textbox(label="請輸入您的內容(如背景、人格特質、個性等)")
    btn = gr.Button("生成自傳 & 修正建議")

    with gr.Row():
        out1 = gr.Textbox(label=" 第一版自傳 (model_writer)")
        out2 = gr.Textbox(label=" 修改建議 (model_reviewer)")
        out3 = gr.Textbox(label=" 第二版自傳 (model_writer 改寫)")

    btn.click(reflect_post, inputs=[user_input], outputs=[out1, out2, out3])

In [ ]:
demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://50b99badf557969bd9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://50b99badf557969bd9.gradio.live
